In [10]:
import logging

from absl import app
from absl import flags
import apache_beam as beam
import xarray as xr
import xarray_beam as xbeam
import xee

import ee
import geemap

import pyproj

import os
import xarray as xr
import geopandas as gp
import pandas as pd
import numpy as np
import sparse

import hvplot.pandas
import hvplot.xarray
import dask
import cf_xarray

from pynhd import NLDI, WaterData
import intake
import cartopy.crs as ccrs
from shapely.geometry import Polygon

import time
import dask
import requests
import dask.array as da
from dask import delayed
from dask.distributed import Client

import requests
from IPython.display import HTML
import warnings
from IPython.display import HTML

from pynhd import NLDI, WaterData
import matplotlib.pyplot as plt
import geopandas as gpd
import contextily as ctx
from scipy.stats import linregress

In [7]:
geemap.ee_initialize()
ee.Initialize()

In [8]:
client = Client()  # Starts the Dask client and connects it to the notebook
client

C:\Users\adi10136\AppData\Local\anaconda3\Lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 63200 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:63200/status,
Dashboard: http://127.0.0.1:63200/status,Workers: 4
Total threads: 8,Total memory: 31.94 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:63201,Workers: 4
Dashboard: http://127.0.0.1:63200/status,Total threads: 8
Started: Just now,Total memory: 31.94 GiB
Comm: tcp://127.0.0.1:63222,Total threads: 2
Dashboard: http://127.0.0.1:63227/status,Memory: 7.98 GiB
Nanny: tcp://127.0.0.1:63204,


# CONUS404

In [16]:
# open the hytest data intake catalog
hytest_cat = intake.open_catalog("https://raw.githubusercontent.com/hytest-org/hytest/main/dataset_catalog/hytest_intake_catalog.yml")
list(hytest_cat)

# open the conus404 sub-catalog
# cat = hytest_cat['conus404-catalog']
cat = hytest_cat['conus404-catalog']
list(cat)

## Select the dataset you want to read into your notebook and preview its metadata
#  conus404-daily-osn
dataset = 'conus404-daily-osn' 
cat[dataset]

print(f"Reading {dataset} metadata...", end='')
ds = cat[dataset].to_dask().metpy.parse_cf()
print("done")
ds
# ds_conus = ds.ACRAINLSM.sel(time=slice('2010-01-01','2020-12-31')).load()
ds_conus = ds.sel(time=slice('2016-01-01','2022-01-01'))
# ds_conus = ds[['ACRAINLSM']].sel(time=slice('1986-01-01','2020-12-31'))
ds_conus

Reading conus404-daily-osn metadata...done


<xarray.Dataset>
Dimensions:         (y: 1015, x: 1367, time: 2193, bottom_top_stag: 51,
                     bottom_top: 50, soil_layers_stag: 4, x_stag: 1368,
                     y_stag: 1016, snow_layers_stag: 3, snso_layers_stag: 7)
Coordinates:
    lat             (y, x) float32 dask.array<chunksize=(350, 350), meta=np.ndarray>
    lon             (y, x) float32 dask.array<chunksize=(350, 350), meta=np.ndarray>
  * time            (time) datetime64[ns] 2016-01-01 2016-01-02 ... 2022-01-01
  * x               (x) float64 -2.732e+06 -2.728e+06 ... 2.728e+06 2.732e+06
  * y               (y) float64 -2.028e+06 -2.024e+06 ... 2.024e+06 2.028e+06
    metpy_crs       object Projection: lambert_conformal_conic
    lat_u           (y, x_stag) float32 dask.array<chunksize=(350, 175), meta=np.ndarray>
    lon_u           (y, x_stag) float32 dask.array<chunksize=(350, 175), meta=np.ndarray>
    lat_v           (y_stag, x) float32 dask.array<chunksize=(175, 350), meta=np.ndarray>
    lon_v           (y_stag, x) float32 dask.array<chunksize=(175, 350), meta=np.ndarray>
Dimensions without coordinates: bottom_top_stag, bottom_top, soil_layers_stag,
                                x_stag, y_stag, snow_layers_stag,
                                snso_layers_stag
Data variables: (12/153)
    ACDEWC          (time, y, x) float32 dask.array<chunksize=(7, 350, 350), meta=np.ndarray>
    ACDRIPR         (time, y, x) float32 dask.array<chunksize=(7, 350, 350), meta=np.ndarray>
    ACDRIPS         (time, y, x) float32 dask.array<chunksize=(7, 350, 350), meta=np.ndarray>
    ACECAN          (time, y, x) float32 dask.array<chunksize=(7, 350, 350), meta=np.ndarray>
    ACEDIR          (time, y, x) float32 dask.array<chunksize=(7, 350, 350), meta=np.ndarray>
    ACETLSM         (time, y, x) float32 dask.array<chunksize=(7, 350, 350), meta=np.ndarray>
    ...              ...
    ZNU             (bottom_top) float32 dask.array<chunksize=(50,), meta=np.ndarray>
    ZNW             (bottom_top_stag) float32 dask.array<chunksize=(51,), meta=np.ndarray>
    ZS              (soil_layers_stag) float32 dask.array<chunksize=(4,), meta=np.ndarray>
    ZSNSO           (time, snso_layers_stag, y, x) float32 dask.array<chunksize=(7, 7, 350, 350), meta=np.ndarray>
    ZWT             (time, y, x) float32 dask.array<chunksize=(7, 350, 350), meta=np.ndarray>
    crs             int64 ...
Attributes: (12/148)
    AER_ANGEXP_OPT:                  1
    AER_ANGEXP_VAL:                  1.2999999523162842
    AER_AOD550_OPT:                  1
    AER_AOD550_VAL:                  0.11999999731779099
    AER_ASY_OPT:                     1
    AER_ASY_VAL:                     0.8999999761581421
    ...                              ...
    WEST-EAST_PATCH_START_STAG:      1
    WEST-EAST_PATCH_START_UNSTAG:    1
    W_DAMPING:                       1
    YSU_TOPDOWN_PBLMIX:              0
    history:                         Tue Mar 29 16:35:22 2022: ncrcat -A -vW ...
    history_of_appended_files:       Tue Mar 29 16:35:22 2022: Appended file ...

In [11]:
ds_conus.crs

<xarray.DataArray 'crs' ()>
[1 values with dtype=int64]
Coordinates:
    metpy_crs  object Projection: lambert_conformal_conic
Attributes: (12/16)
    crs_wkt:                        PROJCRS["unknown",BASEGEOGCRS["unknown",D...
    false_easting:                  0.0
    false_northing:                 0.0
    geographic_crs_name:            unknown
    grid_mapping_name:              lambert_conformal_conic
    horizontal_datum_name:          unknown
    ...                             ...
    prime_meridian_name:            Greenwich
    projected_crs_name:             unknown
    reference_ellipsoid_name:       unknown
    semi_major_axis:                6370000.0
    semi_minor_axis:                6370000.0
    standard_parallel:              [30.0, 50.0]

In [17]:
crs_info = ds_conus.crs
xx = ds_conus.x.values
yy = ds_conus.y.values
globe = ccrs.Globe(ellipse='sphere', semimajor_axis=6370000, semiminor_axis=6370000)
lcc = ccrs.LambertConformal(globe=globe,
                            central_longitude=crs_info.longitude_of_central_meridian, 
                            central_latitude=crs_info.latitude_of_projection_origin,
                            standard_parallels=crs_info.standard_parallel)
print(xx)
print(yy)

[-2732000. -2728000. -2724000. ...  2724000.  2728000.  2732000.]
[-2028000. -2024000. -2020000. ...  2020000.  2024000.  2028000.]


In [18]:
lcc_wkt = lcc.to_wkt()
lcc_wkt

'PROJCRS["unknown",BASEGEOGCRS["unknown",DATUM["unknown",ELLIPSOID["unknown",6370000,0,LENGTHUNIT["metre",1,ID["EPSG",9001]]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8901]]],CONVERSION["unknown",METHOD["Lambert Conic Conformal (2SP)",ID["EPSG",9802]],PARAMETER["Latitude of false origin",39.1000061035156,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8821]],PARAMETER["Longitude of false origin",-97.9000015258789,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8822]],PARAMETER["Latitude of 1st standard parallel",30,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8823]],PARAMETER["Latitude of 2nd standard parallel",50,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8824]],PARAMETER["Easting at false origin",0,LENGTHUNIT["metre",1],ID["EPSG",8826]],PARAMETER["Northing at false origin",0,LENGTHUNIT["metre",1],ID["EPSG",8827]]],CS[Cartesian,2],AXIS["(E)",east,ORDER[1],LENGTHUNIT["metre",1,ID["EPSG",9001]]],AXIS["(N)",north,ORDER[2],LENGTHUNIT["metre",1,ID[

In [50]:
# ds_Pr = ds_conus['ACRAINLSM'].rio.write_crs(ds_conus.crs.metpy.cartopy_crs).drop_vars(['lat', 'lon']).rio.reproject('EPSG:4326')
ds_Pr = ds_conus['ACRAINLSM'].resample(time="Y", loffset="Y").sum()
ds_Pr = ds_Pr.rio.write_crs(lcc_wkt).drop_vars(['lat', 'lon'])
ds_Pr


C:\Users\adi10136\AppData\Local\Temp\ipykernel_230404\162259616.py:2: FutureWarning: Following pandas, the `loffset` parameter to resample will be deprecated in a future version of xarray.  Switch to using time offset arithmetic.
  ds_Pr = ds_conus['ACRAINLSM'].resample(time="Y", loffset="Y").sum()


<xarray.DataArray 'ACRAINLSM' (time: 7, y: 1015, x: 1367)>
dask.array<stack, shape=(7, 1015, 1367), dtype=float32, chunksize=(1, 350, 350), chunktype=numpy.ndarray>
Coordinates:
  * x          (x) float64 -2.732e+06 -2.728e+06 ... 2.728e+06 2.732e+06
  * y          (y) float64 -2.028e+06 -2.024e+06 ... 2.024e+06 2.028e+06
    metpy_crs  object Projection: lambert_conformal_conic
  * time       (time) datetime64[ns] 2017-12-31 2018-12-31 ... 2023-12-31
    crs        int32 0
Attributes:
    description:         acccumlated RAINLSM
    grid_mapping:        crs
    integration_length:  24-hour accumulation
    long_name:           Accumulated liquid precipitation into land surface m...
    units:               mm

In [49]:
ds_Pr.crs

<xarray.DataArray 'crs' ()>
array(0)
Coordinates:
    metpy_crs  object Projection: lambert_conformal_conic
    crs        int32 0
Attributes: (12/17)
    crs_wkt:                        PROJCS["unknown",GEOGCS["unknown",DATUM["...
    semi_major_axis:                6370000.0
    semi_minor_axis:                6370000.0
    inverse_flattening:             0.0
    reference_ellipsoid_name:       unknown
    longitude_of_prime_meridian:    0.0
    ...                             ...
    standard_parallel:              (30.0, 50.0)
    latitude_of_projection_origin:  39.1000061035156
    longitude_of_central_meridian:  -97.9000015258789
    false_easting:                  0.0
    false_northing:                 0.0
    spatial_ref:                    PROJCS["unknown",GEOGCS["unknown",DATUM["...

In [63]:
ds_Pr.hvplot.quadmesh(x='x', y='y', rasterize=True, geo=True, tiles='OSM', cmap='viridis').opts('Image', alpha=0.6)

:DynamicMap   [time]
   :Overlay
      .WMTS.I  :WMTS   [Longitude,Latitude]
      .Image.I :Image   [x,y]   (ACRAINLSM)

In [ ]:
# print(ds_Pr['x'].values, ds_Pr['y'].values)
print(ds_Pr.min().values, ds_Pr.max().values)


In [ ]:
# Replace fill values with NaN before reprojection
ds_Pr_cleaned = ds_Pr.where(ds_Pr != 0, other=np.nan)

In [ ]:
ds_Pr_reproj = ds_Pr.rio.reproject('EPSG:4326')

In [ ]:
ds_Pr_reproj
# print(ds_Pr_reproj['x'].values, ds_Pr_reproj['y'].values)
# print(ds_Pr_reproj.min().values, ds_Pr_reproj.max().values)


ds_Pr_C = (ds_conus['ACRAINLSM'].resample(time="Y", loffset="Y").sum())
ds_Pr_C.rio.write_crs(ds_conus.crs.metpy.cartopy_crs).drop_vars(['lat', 'lon']).rio.reproject('EPSG:4326')

In [38]:
ds_Pr_reproj.rio.crs

CRS.from_epsg(4326)

In [ ]:
ds_Pr_reproj.hvplot.quadmesh(x='x', y='y', rasterize=True, geo=True, tiles='OSM', cmap='viridis').opts('Image', alpha=0.6)